In [1]:
import pandas as pd
import numpy as np
from time import time

In [ ]:
df = pd.read_csv('../cleaned-data/production/transactions.csv', encoding='iso-8859-1', sep='\t')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19332358 entries, 0 to 19332357
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   COCO    object
 1   COMAX   object
 2   COPRO   int64 
dtypes: int64(1), object(2)
memory usage: 442.5+ MB


In [4]:
df = df.groupby('COMAX')['COPRO'].value_counts()

In [5]:
df = pd.DataFrame(df)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2366436 entries, ('00000226d8', 812) to ('ffffd39fd9', 4545)
Data columns (total 1 columns):
 #   Column  Dtype
---  ------  -----
 0   COPRO   int64
dtypes: int64(1)
memory usage: 33.7+ MB


In [7]:
df.index

MultiIndex([('00000226d8',  812),
            ('00000226d8',  954),
            ('00000226d8', 4545),
            ('00005d4463',   53),
            ('00005d4463',  137),
            ('00005d4463',  812),
            ('000093046d',  160),
            ('0000b441aa', 2076),
            ('0000b441aa',  812),
            ('0000b441aa',   53),
            ...
            ('ffff84fdee', 2076),
            ('ffff84fdee',  160),
            ('ffff84fdee',  954),
            ('ffffd39fd9',  812),
            ('ffffd39fd9',   53),
            ('ffffd39fd9',  100),
            ('ffffd39fd9',  160),
            ('ffffd39fd9',  753),
            ('ffffd39fd9',  954),
            ('ffffd39fd9', 4545)],
           names=['COMAX', 'COPRO'], length=2366436)

In [8]:
df.values

array([[12],
       [ 1],
       [ 1],
       ...,
       [ 1],
       [ 1],
       [ 1]])

## Lokking to transpose the data to calculate the PCA values, but couldn't do it in direct way, therefore I had to do all the following JOB

In [9]:
# seperate the data
COMAX, COPRO = [i[0] for i in df.index], [i[1] for i in df.index]
COUNTS = [i[0] for i in df.values]

# get the list of the products
PRODUCTS = set(COPRO)

In [10]:
group = list(zip(COMAX, COPRO, COUNTS))

In [11]:
group

[('00000226d8', 812, 12),
 ('00000226d8', 954, 1),
 ('00000226d8', 4545, 1),
 ('00005d4463', 53, 1),
 ('00005d4463', 137, 1),
 ('00005d4463', 812, 1),
 ('000093046d', 160, 1),
 ('0000b441aa', 2076, 20),
 ('0000b441aa', 812, 4),
 ('0000b441aa', 53, 1),
 ('0000b441aa', 100, 1),
 ('0000b441aa', 137, 1),
 ('0000b441aa', 160, 1),
 ('0000b441aa', 753, 1),
 ('0000b441aa', 954, 1),
 ('0000cf2c88', 2076, 19),
 ('0000cf2c88', 812, 13),
 ('0000cf2c88', 53, 2),
 ('0000cf2c88', 647, 2),
 ('0000cf2c88', 753, 1),
 ('0000d56250', 2076, 9),
 ('0000f6fbc1', 812, 3),
 ('00018e236f', 160, 1),
 ('00018e236f', 954, 1),
 ('00018f7957', 160, 1),
 ('00018f7957', 954, 1),
 ('00018f7957', 4545, 1),
 ('0001936a01', 812, 111),
 ('0001936a01', 100, 1),
 ('0001936a01', 160, 1),
 ('0001936a01', 647, 1),
 ('0001936a01', 954, 1),
 ('0001936a01', 4545, 1),
 ('0001b25677', 2076, 19),
 ('0001b25677', 812, 2),
 ('0001b25677', 100, 1),
 ('0001b25677', 137, 1),
 ('0001b25677', 160, 1),
 ('0001b25677', 954, 1),
 ('0001b25677'

In [12]:
# Make it a data row look a like
def clit(group):
    lol = len(group)
    count = 1
    out = dict()
    
    curr = group[0][0]
    out[curr] = [list(group[0][1:])]
    
    while count < lol:
        if group[count][0] == curr:
            out[curr].append(list(group[count][1:]))
        else:
            curr = group[count][0]
            out[curr] = [list(group[count][1:])]
            
        count += 1
    
    return out

In [13]:
out = clit(group)
out['00000226d8']

[[812, 12], [954, 1], [4545, 1]]

In [14]:
# compelete the data about the missing products, aka their value is equal to 0
def complete(out, PRODUCTS):
    for key in out.keys():
        local = [i[0] for i in out[key]]
        nice = [i for i in PRODUCTS if i not in local]
        
        for i in nice:
            out[key].append([i, 0])

    return out

In [15]:
# The work is done
out = complete(out, PRODUCTS)
out['00000226d8']

[[812, 12],
 [954, 1],
 [4545, 1],
 [160, 0],
 [100, 0],
 [647, 0],
 [137, 0],
 [753, 0],
 [53, 0],
 [2076, 0]]

In [16]:
# to insure we've the same products order we'll sort the lists
def sort_out(out):
    for key in out.keys():
        out[key].sort(key=lambda x: x[0])
        
    return out

In [17]:
# Done
out = sort_out(out)
out['00000226d8']

[[53, 0],
 [100, 0],
 [137, 0],
 [160, 0],
 [647, 0],
 [753, 0],
 [812, 12],
 [954, 1],
 [2076, 0],
 [4545, 1]]

In [18]:
# convert the products numbers to str
def prod_to_str(out):
    for key in out.keys():
        for new in out[key]:
            lol = str(new[0])
            new[0] = '0'*(5 - len(lol)) + lol
        
    return out

In [19]:
# Done
out = prod_to_str(out)
out['00000226d8']

[['00053', 0],
 ['00100', 0],
 ['00137', 0],
 ['00160', 0],
 ['00647', 0],
 ['00753', 0],
 ['00812', 12],
 ['00954', 1],
 ['02076', 0],
 ['04545', 1]]

## Now reassemnling the data but the products as colums as well

In [20]:
# Get the keys
COMAX_ = list(out.keys())
COMAX_[0], COMAX_[-1]

('00000226d8', 'ffffd39fd9')

In [21]:
# Products, already converted them to strings
PRODUCTS = [i[0] for i in out['00000226d8']]

In [26]:
data = dict()
data['COMAX'] = COMAX_
for i in PRODUCTS:
    data[i] = list()

In [29]:
data.keys()

dict_keys(['COMAX', '00053', '00100', '00137', '00160', '00647', '00753', '00812', '00954', '02076', '04545'])

In [31]:
# reassamble all the products values
def done(data, out):
    for key in out.keys():
        for val in out[key]:
            data[val[0]].append(val[1])
        
    return data

In [32]:
data = done(data, out)

In [33]:
for key in data.keys():
    print(len(data[key]))

570180
570180
570180
570180
570180
570180
570180
570180
570180
570180
570180


In [34]:
data = pd.DataFrame(data)

In [35]:
data.head()

,COMAX,00053,00100,00137,00160,00647,00753,00812,00954,02076,04545
0,00000226d8,0,0,0,0,0,0,12,1,0,1
1,00005d4463,1,0,1,0,0,0,1,0,0,0
2,000093046d,0,0,0,1,0,0,0,0,0,0
3,0000b441aa,1,1,1,1,0,1,4,1,20,0
4,0000cf2c88,2,0,0,0,2,1,13,0,19,0


In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570180 entries, 0 to 570179
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   COMAX   570180 non-null  object
 1   00053   570180 non-null  int64 
 2   00100   570180 non-null  int64 
 3   00137   570180 non-null  int64 
 4   00160   570180 non-null  int64 
 5   00647   570180 non-null  int64 
 6   00753   570180 non-null  int64 
 7   00812   570180 non-null  int64 
 8   00954   570180 non-null  int64 
 9   02076   570180 non-null  int64 
 10  04545   570180 non-null  int64 
dtypes: int64(10), object(1)
memory usage: 47.9+ MB


In [51]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [48]:
x = data.loc[:, PRODUCTS].values

In [49]:
x = StandardScaler().fit_transform(x)

In [56]:
pca = PCA(n_components=4)

In [71]:
principalComponents = pca.fit_transform(x)

In [77]:
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['PCA-1', 'PCA-2', 'PCA-3', 'PCA-4'])

In [78]:
principalDf

,PCA-1,PCA-2,PCA-3,PCA-4
0,-0.666116,-1.136881,0.978239,0.541362
1,-1.288706,0.000403,-0.677441,-0.562691
2,-1.690757,-0.114129,-0.182771,0.196209
3,1.975899,0.927151,-1.653565,-1.814180
4,0.255077,2.575486,-2.749994,0.749088
...,...,...,...,...
570175,-0.373618,-0.564755,0.611408,-1.057193
570176,-0.080382,-0.577872,1.138760,-1.225243
570177,-2.153333,0.145627,0.174661,0.354602
570178,-0.898056,-0.405393,0.073852,0.055392


In [79]:
#principalDf['COMAX'] = data['COMAX']
principalDf.insert(0, 'COMAX', data['COMAX'], True)

In [81]:
principalDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570180 entries, 0 to 570179
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   COMAX   570180 non-null  object 
 1   PCA-1   570180 non-null  float64
 2   PCA-2   570180 non-null  float64
 3   PCA-3   570180 non-null  float64
 4   PCA-4   570180 non-null  float64
dtypes: float64(4), object(1)
memory usage: 21.8+ MB


In [83]:
principalDf.to_csv("PCA-data.csv", index=False, sep="\t")